## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- 위키피디아에서 데이터를 가져와 Qdrant를 활용한 RAG 검색 시스템 구축
- 벡터 검색을 사용하여 GPT 모델이 문서를 기반으로 더 정확한 응답을 생성

## 왜 필요한가?
- 문서를 벡터로 변환하여 유사도 기반 검색을 수행할 수 있음
- LLM 기반 질의응답 시스템의 정확도를 향상시킴

## 주요 개념:
- **RAG (Retrieval-Augmented Generation)**: 검색된 문서를 바탕으로 AI 모델이 답변을 생성하는 기술
- **위키피디아 데이터 로드**: 공공 데이터를 활용하여 RAG 성능 테스트 가능

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [2]:
# Qdrant 클라이언트 설정
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "ex13_wiki_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if collection_name not in qdrant_client.get_collections().collections:
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 MiniLM-L6-v2 임베딩 차원 수
    )

In [3]:
# LangChain 임베딩 설정
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_169301/3947042695.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [4]:
# 위키피디아 문서 로드
loader = WikipediaLoader(query="Vector database", lang="en")
docs = loader.load()

/home/psbreeze/billionvectors/study/study_inflearn_dev/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/psbreeze/billionvectors/study/study_inflearn_dev/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [5]:
# Qdrant 벡터 저장소 설정
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=embeddings
)

# 위키피디아 문서 임베딩 후 Qdrant에 저장
vectorstore.add_documents(docs)

/tmp/ipykernel_169301/3955492599.py:2: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


['5227b6957c2744c3afb8d1592deea269',
 'dcff8d12a9c241a9bce3a47523f60c98',
 'b4fcb7175813499d89d1dcfeb7de367e',
 '115c7a62da0c486bab75b63fa0cbccf1',
 '850598bbb1c744b698ee6640111c752f',
 '4e3e34ddca72448d92da9124774261e3',
 '3273f569288a4e8d9866eeae52f50512',
 'ce9fd3915ef14de08040a3c861ebecda',
 '4fe34465c5164ed6b0d15afee46bf5e2',
 'c5726888e5024d11a07f55a434d97877',
 'b7e7d44afb32416394e6a3373b7e8ef8',
 'ef51bc5a86e84d649878c0e0992c9c5e',
 'dfb3cc6d1b4248dbb08eebd33aba2929',
 '2e9b0859580a42289832ac3ac29957f9',
 '36d136760a1947fab884886cd1ae2da6',
 'bdc9c0d6257d4ebbaed036469c331de6',
 '6969e502084d4eb4be4a42bdf501820f',
 '34088152af8746cebf621bc8d321a9f9',
 '415811e87e924033a30ba86f13f9f336',
 'd5cadbd791a0417ea9c0b510fd5cd256',
 '03eb43e7f7624ea28d953e1017fc524c',
 '4c17bc80567e49feb1d5b111b37f0a9c',
 '30b6cbb4553547d4a968e8ebfe82ed37',
 '49effa5a44534e0eba83b4893dac03ae']

In [6]:
# OpenAI API 키 설정 (환경 변수에서 가져오거나 기본값 설정)
openai_api_key = os.environ.get("OPENAI_API_KEY", "your_api_key")

# OpenAI LLM 설정
llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key)

/tmp/ipykernel_169301/2245453385.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key)


In [7]:
# RAG 검색을 위한 체인 생성
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff", "map_reduce", "refine" 중 선택 가능
    retriever=retriever
)


In [8]:
question = "What is a vector database?"
response = qa_chain.run(question)
print("LLM 응답:", response)

/tmp/ipykernel_169301/3769801674.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)


LLM 응답:  A vector database is a type of database that can store and search for data in the form of vectors, which are mathematical representations of data in a high-dimensional space. These databases are commonly used for similarity search, semantic search, multi-modal search, recommendation engines, and other applications that involve finding similar or related data items. They typically implement advanced algorithms, such as Approximate Nearest Neighbor algorithms, to efficiently search for and retrieve relevant data. Vector databases are also used in conjunction with machine learning methods to compute feature vectors and improve the performance of large language models. 
